# Gamut Mapping - Compression Functions 01

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'figure.figsize': (11, 11),
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-141-g3bebd7e9                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Compression & Utility Functions

In [3]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * np.tanh((x - a) / b), x)


def atan_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * 2 / np.pi * np.arctan(((np.pi / 2) * (x - a)) / b), x)


def power_compression_function(x, a=0.8, b=1 - 0.8, p=2.0):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + (((x - a) / b) / np.power(1 + np.power((x - a) / b, p), 1.0/p)) * b, x)


def simple_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + (-1 / ((x - a) / b + 1) + 1) * b, x)


def derivative(x, func, d=1e-7):
    return (func(x + d) - func(x - d)) / (2 * d)


In [4]:
colour.plotting.plot_multi_functions(
    {
        'tanh': tanh_compression_function,
        'd/dx(tanh)': lambda x: derivative(x, tanh_compression_function),
        'atan': atan_compression_function,
        'd/dx(atan)': lambda x: derivative(x, atan_compression_function),
        'power': power_compression_function,
        'd/dx(power)': lambda x: derivative(x, power_compression_function),
        'simple': simple_compression_function,
        'd/dx(simple)': lambda x: derivative(x, simple_compression_function),
    },
    **{
        'figure.figsize': (11, 11),
        'bounding_box': (0.5, 1.75, 0, 1.25),
        'samples':
        np.linspace(0, 2, 1000),
        'plot_kwargs': [
            {
                'c': 'r',
                'linestyle': 'dashdot'
            },
            {
                'c': 'r'
            },
            {
                'c': 'g',
                'linestyle': 'dashdot'
            },
            {
                'c': 'g'
            },
            {
                'c': 'b',
                'linestyle': 'dashdot'
            },
            {
                'c': 'b'
            },
            {
                'c': 'y',
                'linestyle': 'dashdot'
            },
            {
                'c': 'y'
            },
        ]
    },
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## TANH - Power Fitting

In [5]:
SAMPLES = np.linspace(0, 8, 1000)
TANH_SAMPLES = tanh_compression_function(SAMPLES)

POPT, PCOV = scipy.optimize.curve_fit(
    power_compression_function, SAMPLES, TANH_SAMPLES, maxfev=50000)

print(POPT)
print(
    colour.utilities.metric_mse(TANH_SAMPLES,
                                power_compression_function(SAMPLES, *POPT)))

colour.plotting.artist()
plt.plot(SAMPLES, TANH_SAMPLES, 'r', label='tanh')
plt.plot(SAMPLES, power_compression_function(SAMPLES, *POPT), 'g', label='power')
plt.legend()
plt.show()

[ 0.65953861  0.34051662  4.6993435 ]
7.53150995846e-08


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## ATAN - Power Fitting

In [6]:
ATAN_SAMPLES = atan_compression_function(SAMPLES)

POPT, PCOV = scipy.optimize.curve_fit(
    power_compression_function, SAMPLES, ATAN_SAMPLES, maxfev=50000)

print(POPT)
print(
    colour.utilities.metric_mse(ATAN_SAMPLES,
                                power_compression_function(SAMPLES, *POPT)))

colour.plotting.artist()
plt.plot(SAMPLES, ATAN_SAMPLES, 'r', label='atan')
plt.plot(SAMPLES, power_compression_function(SAMPLES, *POPT), 'g', label='power')
plt.legend()
plt.show()

[ 0.83715264  0.16187046  1.18238345]
9.2282913999e-08


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Solving Intersection with Y = 1

In [7]:
X, A, I = 1.2, 0.8, 1

figure, axes = colour.plotting.plot_multi_functions(
    {
        'tanh': lambda x: tanh_compression_function(
            x, A, scipy.optimize.fsolve(
                lambda x: tanh_compression_function(X, A, x) - I, 0.5)),
        'simple': lambda x: simple_compression_function(
            x, A, scipy.optimize.fsolve(
                lambda x: simple_compression_function(X, A, x) - I, 0.5)),
    },
    **{
        'standalone': False,
        'bounding_box': [0, 2, 0, 2],
        'samples':
        np.linspace(0, 2, 1000),
        'plot_kwargs': [
            {
                'c': 'r',
            },
            {
                'c': 'g',
            },
        ]
    },
)

axes.scatter(X, I, c='b', s=50, zorder=4);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Maximum Representable Value

In [8]:
HFD = [1]

while True:
    HFD.append(np.nextafter(HFD[-1], +np.inf, dtype=np.float16))
    if HFD[-1] > 65504:
        break

print(HFD[:10])

[1, 1.0009766, 1.0019531, 1.0029297, 1.0039062, 1.0048828, 1.0058594, 1.0068359, 1.0078125, 1.0087891]


In [9]:
for t in np.linspace(0, 1, 11):
    print(f'[ Threshold {np.around(t, 1)} ]')
    for func in (tanh_compression_function, atan_compression_function, simple_compression_function):
        print(func.__name__.split('_')[0],
              HFD[np.argmax((func(HFD[:-1], t, 1 - t) - func(HFD[1:], t, 1 - t)).astype(np.float16))])
    print('\n')

[ Threshold 0.0 ]
tanh 6.58594
atan 7376.0
simple 23168.0


[ Threshold 0.1 ]
tanh 6.02734
atan 6636.0
simple 14744.0


[ Threshold 0.2 ]
tanh 5.46875
atan 5900.0
simple 13104.0


[ Threshold 0.3 ]
tanh 4.91016
atan 3650.0
simple 8108.0


[ Threshold 0.4 ]
tanh 4.35156
atan 3130.0
simple 6952.0


[ Threshold 0.5 ]
tanh 3.61914
atan 1844.0
simple 5792.0


[ Threshold 0.6 ]
tanh 3.0957
atan 1476.0
simple 3278.0


[ Threshold 0.7 ]
tanh 2.57227
atan 783.0
simple 1738.0


[ Threshold 0.8 ]
tanh 1.97852
atan 369.5
simple 820.0


[ Threshold 0.9 ]
tanh 1.48926
atan 93.0625
simple 205.625


[ Threshold 1.0 ]
tanh 1
atan 1
simple 1


